In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [ ]:
dataset_path = "/content/drive/MyDrive/ColabNote/data set"

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,        # Normalize ค่าพิกเซล (0-255 → 0-1)
    validation_split=0.2    # แบ่ง 80% train / 20% test
)
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(299, 299),
    batch_size=32,
    class_mode="binary",
    subset="training"
)

Found 1280 images belonging to 2 classes.


In [ ]:
val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(299, 299),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)

Found 320 images belonging to 2 classes.


In [ ]:
base_model = Xception(weights="imagenet", include_top=False, input_shape=(299, 299, 3))

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation="relu")(x)
output = Dense(1, activation="sigmoid")(x)  # Binary Classification

model = Model(inputs=base_model.input, outputs=output)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model.fit(train_generator, epochs=3, validation_data=val_generator)

Epoch 1/3
40/40 ━━━━━━━━━━━━━━━━━━━━ 591s 15s/step - accuracy: 0.9121 - loss: 0.2201 - val_accuracy: 0.8656 - val_loss: 0.3063
Epoch 2/3
40/40 ━━━━━━━━━━━━━━━━━━━━ 533s 13s/step - accuracy: 0.9740 - loss: 0.0678 - val_accuracy: 0.8750 - val_loss: 0.3181
Epoch 3/3
40/40 ━━━━━━━━━━━━━━━━━━━━ 527s 13s/step - accuracy: 0.9934 - loss: 0.0301 - val_accuracy: 0.9125 - val_loss: 0.2336


In [ ]:
test_loss, test_acc = model.evaluate(val_generator)
print(f"Accuracy for Xception: {test_acc:.1%}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 119s 11s/step - accuracy: 0.9093 - loss: 0.2279
Accuracy for Xception: 91.3%
